In [40]:
%matplotlib inline
import numpy as np
import scipy.integrate
import scipy.interpolate
import matplotlib.pyplot as plt

In [41]:
def get_NofZ_unnormed(dNdzpar, dNdztype, z):
    """ Returns the dNdz of the sources as a function of spec-z."""

    #z = scipy.linspace(z_min+0.0001, z_max, zpts)

    if (dNdztype == 'Nakajima'):
        # dNdz takes form like in Nakajima et al. 2011 equation 3
        a = dNdzpar[0]
        zs = dNdzpar[1]
        nofz_ = (z / zs)**(a-1) * np.exp( -0.5 * (z / zs)**2)
    elif (dNdztype == 'Smail'):
        # dNdz take form like in Smail et al. 1994
        alpha = dNdzpar[0]
        z0 = dNdzpar[1]
        beta = dNdzpar[2]
        nofz_ = z**alpha * np.exp( - (z / z0)**beta)
    else:
        print "dNdz type "+str(dNdztype)+" not yet supported; exiting."
        exit()

    return nofz_

In [42]:
def p_z(z_ph, z_sp, pzpar, pztype):
    """ Returns the probability of finding a photometric redshift z_ph given that the true redshift is z_sp. """
    
    if (pztype == 'Gaussian'):
        sigz = pzpar[0]
        p_z_ = np.exp(-(z_ph - z_sp)**2 / (2.*(sigz*(1.+z_sp))**2)) / (np.sqrt(2.*np.pi)*(sigz*(1.+z_sp)))
    else:
        print "Photo-z probability distribution "+str(pztype)+" not yet supported; exiting."
        exit()
    
    return p_z_

In [43]:
def com(z_, OmC, OmB, HH0, Nnu):
    """ Gets the comoving distance in units of Mpc/h at a given redshift, z_. """
    
    OmR	=	2.47*10**(-5)/(HH0/100.)**2
    OmN	=	Nnu*(7./8.)*(4./11.)**(4./3.)*OmR
    OmL = 1. - OmC - OmB - OmR - OmN
    c=2.99792458*10**(8)
    H0	=	10**(5)/c
    
    def chi_int(z):
        return 1. / (H0 * ( (OmC+OmB)*(1+z)**3 + OmL + (OmR+OmN) * (1+z)**4 )**(0.5))

    if hasattr(z_, "__len__"):
        chi=np.zeros((len(z_)))
        for zi in range(0,len(z_)):
            #print "zi in com=", zi
            chi[zi] = scipy.integrate.quad(chi_int,0,z_[zi])[0]
    else:
        chi = scipy.integrate.quad(chi_int, 0, z_)[0]

    return chi

In [44]:
def z_interpof_com(OmC, OmB, HH0, Nnu):
    """ Returns an interpolating function which can give z as a function of comoving distance. """

    z_vec = scipy.linspace(0., 10., 10000) # This hardcodes that we don't care about anything over z=100.

    com_vec = com(z_vec, OmC, OmB, HH0, Nnu)

    z_of_com = scipy.interpolate.interp1d(com_vec, z_vec)
    com_of_z =  scipy.interpolate.interp1d(z_vec, com_vec)

    return	(z_of_com, com_of_z)

In [45]:
def sigma_e(z_s_):
    """ Returns a value for the model for the per-galaxy noise as a function of source redshift"""

    if (survey=='SDSS'):

        if hasattr(z_s_, "__len__"):
            sig_e = 2. / S_to_N * np.ones(len(z_s_))
        else:
            sig_e = 2. / S_to_N

    elif(survey=='LSST_DESI'):
        if hasattr(z_s_, "__len__"):
            sig_e = a_sm / SN_med * ( 1. + (b_sm / R_med)**c_sm) * np.ones(len(z_s_))
        else:
            sig_e = a_sm / SN_med * ( 1. + (b_sm / R_med)**c_sm) 

    return sig_e

def get_SigmaC_inv(z_s_, z_l_):
    """ Returns the theoretical value of 1/Sigma_c, (Sigma_c = the critcial surface mass density) """

    com_s = chi_of_z(z_s_) 
    com_l = chi_of_z(z_l_) 

    # Get scale factors for converting between angular-diameter and comoving distances.
    a_l = 1. / (z_l_ + 1.)
    a_s = 1. / (z_s_ + 1.)
    
    D_s = a_s * com_s # Angular diameter source distance.
    D_l = a_l * com_l # Angular diameter lens distance
    D_ls = (D_s - D_l) 
        
    # Units are pc^2 / (h Msun), comoving
    Sigma_c_inv = 4. * np.pi * (Gnewt * Msun) * (10**12 / c**2) / mperMpc *   D_l * D_ls * (1 + z_l_)**2 / D_s

    if hasattr(z_s_, "__len__"):
        for i in range(0,len(z_s_)):
            if(z_s_[i]<=z_l_):
                Sigma_c_inv[i] = 0.
    else:
        if (z_s_<=z_l_):
            Sigam_c_inv = 0.

    return Sigma_c_inv


def weights(e_rms, z_, z_l_):

    """ Returns the inverse variance weights as a function of redshift. """

    SigC_t_inv = get_SigmaC_inv(z_, z_l_)

    weights = SigC_t_inv**2/(sigma_e(z_)**2 + e_rms**2 * np.ones(len(z_)))

    return weights

In [46]:
# Set up interpolating functions for z(chi) and chi(z)
(z_of_chi, chi_of_z) = z_interpof_com(0.2, 0.05, 70., 3.046)

# Constants / conversions
mperMpc = 3.0856776*10**22
Msun = 1.989*10**30 # in kg
Gnewt = 6.67408*10**(-11)
c=2.99792458*10**(8)

In [50]:
# Set some parameters
zeff = 0.77
pztype = 'Gaussian'; pzpar = [0.05]
dNdztype='Smail'; dNdzpar =  [1.24, 0.51, 1.01]
rp_fix = 1.0
zsmax = 6.0

zphmin = 0.0
zphmax = 7.0
deltaz = 0.57
close_cut = 100
erms = 0.18

zPi_min = 0.001
zPi_max = 6.0

survey = 'LSST_DESI'
#S_to_N = 15.
a_sm  = 1.58
b_sm  = 5.03
c_sm  = 0.39
SN_med= 11.8
R_med = 1.63

b_l = 3.57
b_s = 2.05

# Import the correlation function, from CAMB w/ halofit + FFTlog
(r, xi_2h) = np.loadtxt('../txtfiles/xi_z='+str(zeff)+'.txt', unpack=True)
xi_2h = b_l* b_s * xi_2h

# Import the 1-halo term as computed in our code
#(r_1h, xi_1h) = np.loadtxt('../txtfiles/xi_gg_1halo_'+survey+'.txt', unpack=True)
#xi_1h_interp = scipy.interpolate.interp1d(r_1h,xi_1h)
#xi_1h = xi_1h_interp(r)
xi = xi_2h #xi_1h + xi_2h


In [51]:
# Get the comoving distance associated to the lens redshift
chi_eff = com(zeff, 0.2, 0.05, 70., 3.046)

# Figure out the min and max value of the * positive * part of the vector of projected distances
if (min(r)>rp_fix):
    minPiPos = np.sqrt(min(r)**2 - rp_fix**2)
else:
    minPiPos = 0.
#if (max(r)<com(zsmax, 0.2, 0.05, 70., 3.046)):
#    maxPiPos = np.sqrt(max(r)**2 - rp_fix**2)
#else:
#    maxPiPos = chi_of_z(zsmax)-chi_eff

maxPiPos = chi_of_z(zPi_max) - chi_eff

Pi_pos = scipy.linspace(minPiPos, maxPiPos, 50000)
    
# Pi can be positive or negative, so now flip this and include the negative values, but only down to z=0
# And avoid including multiple of the same values - this messes up some integration routines.
Pi_pos_vec = list(Pi_pos)[1:]
Pi_pos_vec.reverse()
index_cut = next(j[0] for j in enumerate(Pi_pos_vec) if j[1]<=(chi_eff-chi_of_z(zPi_min)))
print chi_of_z(zPi_min)+chi_eff
Pi = np.append(-np.asarray(Pi_pos_vec[index_cut:]), Pi_pos)

# Get the correlation function in terms of Pi at rp = 1
xi_interp_r = scipy.interpolate.interp1d(r, xi)
xi_ofPi = xi_interp_r(np.sqrt(rp_fix**2 + Pi**2))

# Get the vector of com dist values associated to Pi values:
com_Pi = chi_eff + Pi

# Get the associated z's
z_Pi = z_of_chi(com_Pi)

print "z_Pi=", z_Pi

# Now we effectively have xi_{ls}(rp=1, Pi(z_s); z_eff)

1945.32426314
z_Pi= [  1.02755176e-03   1.05553393e-03   1.08351610e-03 ...,   5.99947919e+00
   5.99973958e+00   6.00000000e+00]


In [52]:
# Okay, now we do the required integrals:
# Define the z_ph vectors for the three subsamples we are about:
lenzph = 10000
z_a = scipy.linspace(zeff, zeff +deltaz, lenzph)
z_b = scipy.linspace(zeff+deltaz, zphmax, lenzph)
# For the "assoc" sample we need to get the z-edges
zasc_min = z_of_chi(chi_eff - close_cut)
zasc_max = z_of_chi(chi_eff + close_cut)
z_asc = scipy.linspace(zasc_min, zasc_max, lenzph)

# Get dNdz
dNdz = get_NofZ_unnormed(dNdzpar, dNdztype, z_Pi)

# Do the integrals in spec-z
specint_num_a = np.zeros(lenzph); specint_num_b = np.zeros(lenzph); specint_num_asc = np.zeros(lenzph)
specint_denom_a = np.zeros(lenzph); specint_denom_b = np.zeros(lenzph); specint_denom_asc = np.zeros(lenzph)
for i in range(0, lenzph):
    specint_num_a[i] = scipy.integrate.simps(dNdz * p_z(z_a[i], z_Pi, pzpar, pztype) * xi_ofPi, z_Pi)
    specint_num_b[i] = scipy.integrate.simps(dNdz * p_z(z_b[i], z_Pi, pzpar, pztype)* xi_ofPi, z_Pi)
    specint_num_asc[i] = scipy.integrate.simps(dNdz * p_z(z_asc[i], z_Pi, pzpar, pztype) * xi_ofPi, z_Pi)
    
    specint_denom_a[i] = scipy.integrate.simps(dNdz * p_z(z_a[i], z_Pi, pzpar, pztype), z_Pi)
    specint_denom_b[i] = scipy.integrate.simps(dNdz * p_z(z_b[i], z_Pi, pzpar, pztype), z_Pi)
    specint_denom_asc[i] = scipy.integrate.simps(dNdz * p_z(z_asc[i], z_Pi, pzpar, pztype), z_Pi)
    
# Now do the integrals in photo-z
w_a = weights(erms,z_a, zeff)
w_b = weights(erms,z_b, zeff)
w_asc = weights(erms,z_asc, zeff)

B_min_1_a = scipy.integrate.simps(w_a * specint_num_a, z_a) / scipy.integrate.simps(w_a* specint_denom_a, z_a)
B_min_1_b = scipy.integrate.simps(w_b * specint_num_b, z_b) / scipy.integrate.simps(w_b* specint_denom_b, z_b)
B_min_1_asc = scipy.integrate.simps(w_asc*specint_num_asc, z_asc) / scipy.integrate.simps(w_asc* specint_denom_asc, z_asc)

print "Boost-1, sample a=", B_min_1_a
print "Boost-1, sample b =", B_min_1_b
print "Boost-1, sample assoc =", B_min_1_asc
    

Boost-1, sample a= 0.0316631727836
Boost-1, sample b = -1.47902991291e-07
Boost-1, sample assoc = 0.758782906799
